

- Total duration to be divided into 2 sections if there is a change from one emotional state to another during the piece
- Each section will be divided into (number of notes), which will be calculated based on the randomly selected range of note durations
- The last note of every section will be atleast 3 seconds long
- The section duration is reduced by a randomly selected note duration within the range

Alternatively:
- If the affect has low arousal, the notes/chords can all be set to a standard duration (3-5s) 
- If affect has high arousal, the duration of each note/chord is randomly sampled from the range of note durations
- This process continues until the final note is less than 3 seconds long, in which case the penultimate note is extended to the end of the audio

In [3]:
import numpy as np
import pandas as pd
from collections import Counter
np.random.seed(42)

# read file
data = pd.read_csv('Liverpool_band_chord_sequence.csv')

In [15]:
n = 3
chords = data['chords'].values
ngrams = zip(*[chords[i:] for i in range(n)])
bigrams = [" ".join(ngram) for ngram in ngrams]

bigrams[:5]

['F Em7 A7', 'Em7 A7 Dm', 'A7 Dm Dm7', 'Dm Dm7 Bb', 'Dm7 Bb C7']

In [30]:
def predict_next_state(chord:str, data:list=bigrams, segment:int=1):
    """Predict next chord based on current state."""
    # create list of bigrams which stats with current chord
    bigrams_with_current_chord = [bigram for bigram in bigrams if bigram.split(' ')[0]==chord]
    # count appearance of each bigram
    count_appearance = dict(Counter(bigrams_with_current_chord))
    # convert apperance into probabilities
    for ngram in count_appearance.keys():
        count_appearance[ngram] = count_appearance[ngram]/len(bigrams_with_current_chord)
    
    # create list of possible options for the next chord
    options = [key.split(' ')[1] for key in count_appearance.keys()]
    # print(options)
    # create  list of probability distribution
    probabilities = list(count_appearance.values())
    # return random prediction
    return np.random.choice(options, p=probabilities)

In [31]:
predict_next_state('F')

'Fsus4'

In [32]:
def generate_sequence(chord:str=None, data:list=bigrams, length:int=30):
    """Generate sequence of defined length."""
    # create list to store future chords
    chords = []
    chords.append(chord)
    for n in range(length):
        # append next chord for the list
        chords.append(predict_next_state(chord, bigrams))
        # use last chord in sequence to predict next chord
        chord = chords[-1]
    return chords

In [33]:
generate_sequence('C')

['C',
 'Dm7',
 'G7',
 'Bb',
 'Dm',
 'Dm7',
 'Bb',
 'Dm',
 'Dm7',
 'Bb',
 'F',
 'Fsus4',
 'F',
 'F',
 'Em7',
 'A7',
 'Dm',
 'Dm7',
 'G7',
 'Bb',
 'Dm',
 'Gm6',
 'C7',
 'F',
 'C',
 'Bb',
 'Dm',
 'Gm6',
 'C7',
 'F',
 'F']